In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_selection import VarianceThreshold
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
from sklearn.decomposition import PCA
# Any results you write to the current directory are saved as output.

import matplotlib.pyplot as plt
from copy import deepcopy
import pandas as pd
import numpy as np
import random
import gc

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score

from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import FastICA
from sklearn.feature_selection import SelectKBest, chi2, SelectFpr
from sklearn.decomposition import NMF
from sklearn.decomposition import PCA,  KernelPCA

from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import neural_network
from sklearn.svm import NuSVC
from sklearn.svm import SVC

from mlxtend.classifier import StackingClassifier
from mlxtend.classifier import EnsembleVoteClassifier
import optuna
import skopt
import mlxtend

['train.csv', 'sample_submission.csv', 'test.csv']


In [2]:
df_train = pd.read_csv('../input/train.csv')
df_test = pd.read_csv('../input/test.csv')
submission = pd.read_csv('../input/sample_submission.csv')

In [3]:
cols = [c for c in df_train.columns if c not in ['id', 'target', 'wheezy-copper-turtle-magic']]

In [4]:
def transform(df_train, df_test, mappers):
    train = df_train[cols].values
    test = df_test[cols].values
    data = np.concatenate((train,test), axis = 0)
    
    for elt in mappers:
        
        mapper = elt.fit(data)
        train = mapper.transform(train)
        test = mapper.transform(test)
        data = np.concatenate((train,test), axis = 0)
    return train, test

In [77]:
ind = 0
df = df_train[df_train['wheezy-copper-turtle-magic']==ind]
y = df['target'].values
X_val = df_test[df_test['wheezy-copper-turtle-magic']==ind]

X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state = 42)

X_train  = VarianceThreshold(threshold=1.5).fit_transform(X_train[cols])
X_test  = VarianceThreshold(threshold=1.5).fit_transform(X_test[cols])

# X_train, X_test = transform(X_train, X_test, mappers)

X_val  = VarianceThreshold(threshold=1.5).fit_transform(X_val[cols])

title = []
for elt in range(X_train.shape[1]):
    title.append('c'+str(elt))

clf = QuadraticDiscriminantAnalysis(reg_param = 0.11)
clf.fit(X_train, y_train)

pred_test_proba = clf.predict_proba(X_test)[:,1]
pred_test = clf.predict(X_test)
pred_val = clf.predict(X_val)
df_training = pd.DataFrame(X_val, columns = title)
df_training['target'] = pred_val
df_training = df_training.loc[:,:]



base_score = roc_auc_score(y_test, pred_test_proba)
print(base_score)
print(confusion_matrix(y_test, pred_test))
train = X_train
y = y_train

for i in range(10):
    train = np.concatenate((X_train, df_training[title]), axis = 0)
    y = np.concatenate((y_train, df_training['target']), axis = 0 )
    
    clf = QuadraticDiscriminantAnalysis(reg_param = 0.11)
    clf.fit(train, y)
    pred_test_proba = clf.predict_proba(X_test)[:,1]
    
    baseline = roc_auc_score(y_test, pred_test_proba)
    
    df_trial = deepcopy(df_training)
    pred_trial = clf.predict(df_training[title])
    clf = QuadraticDiscriminantAnalysis(reg_param = 0.11)
    clf.fit(np.concatenate((X_train, df_trial[title]), axis = 0), np.concatenate((y_train, pred_trial), axis = 0 ))
    
    score = roc_auc_score(y_test, clf.predict_proba(X_test)[:,1])
    if score > baseline:
        df_training['target'] = pred_trial
    print(score)
    
# for i in range(200):
#     print(i)
#     df_val = pd.DataFrame(X_val, columns = title)
#     df_val['target'] = pred_val
    
#     df_sample = df_val.sample(frac = 0.01)
    
#     X_sample = df_sample[title]
#     y_sample = df_sample['target']
    
    
#     train = np.concatenate((train, df_training[title], X_sample), axis = 0)
#     y = np.concatenate((y, df_training['target'],y_sample), axis = 0 )
    
# #     train = np.concatenate((train, X_sample), axis = 0)
# #     y = np.concatenate((y,y_sample), axis = 0 )
    
#     clf = QuadraticDiscriminantAnalysis(reg_param = 0.11)
#     clf.fit(train, y)
    
#     pred_test_proba = clf.predict_proba(X_test)[:,1]
    
    
#     score = roc_auc_score(y_test, pred_test_proba)
    
#     if score > base_score:
#         base_score = score
#         pred_val = clf.predict(X_val)
#         df_training = pd.concat([df_training, df_sample])
        
# #         X_train = np.concatenate((X_train, X_sample), axis = 0)
# #         y_train = np.concatenate((y_train, y_sample), axis = 0)
    
#     df_training = df_training.drop_duplicates(subset = title, keep = False)
#     print(score)
    


0.9629237288135594
[[45  3]
 [ 3 56]]
0.9721045197740114
0.9721045197740114
0.9721045197740114
0.9721045197740114
0.9721045197740114
0.9721045197740114
0.9721045197740114
0.9721045197740114
0.9721045197740114
0.9721045197740114


In [78]:
print(base_score)

0.9629237288135594


In [79]:
df_training = df_training.drop_duplicates(subset = title, keep = False)
X_t = df_training[title].values
y_t = df_training['target']
clf = QuadraticDiscriminantAnalysis(reg_param = 0.11)
clf.fit(np.concatenate((X_t, X_test), axis = 0),np.concatenate((y_t, y_test), axis = 0))

clf.fit(X_t, y_t)
pred = clf.predict_proba(X_train)[:,1]
roc_auc_score(y_train, pred)

0.9456607755209707

In [80]:
df_training.shape

(253, 47)

In [46]:


ind = 0
df = df_train[df_train['wheezy-copper-turtle-magic']==ind]
y = df['target'].values

X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state = 42)
X_val = df_test[df_test['wheezy-copper-turtle-magic']==ind][cols]
mappers = [
    ## Data Augmentation
#             PolynomialFeatures(degree=2),
    ##Data reduction
            VarianceThreshold(threshold=1.5),
#              KernelPCA(n_components = 46 , kernel = 'linear', degree = 2)
#             PCA(n_components=46, whiten = True),
#             FastICA(n_components=46, random_state=0, max_iter = 500),
    
    ## Data Augmentation
#             PolynomialFeatures(degree=2),
    
    ## Scaler
#             StandardScaler(),
#             Normalizer(norm="l2"),
#             Normalizer(norm="l1"),
#             RobustScaler(),
#             MinMaxScaler(),
                        ]
X_train, X_test = transform(X_train, X_test, mappers)
X_val  = VarianceThreshold(threshold=1.5).fit_transform(X_val)

In [42]:
from sklearn.metrics import confusion_matrix
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# clf = LinearDiscriminantAnalysis()
clf = QuadraticDiscriminantAnalysis(reg_param = 0.11)
# clf = NuSVC(probability=True, kernel='poly', degree=2, gamma='auto', random_state=2, nu=0.7, coef0=0.08)

clf.fit(X_train, y_train)  


pred = clf.predict_proba(X_test)[:,1]
pred1 = clf.predict(X_test)

# pred2 = clf.predict(np.concatenate((X_test, X_val), axis = 0))
pred2 = clf.predict(X_val)

print(roc_auc_score(y_test, pred))
print(confusion_matrix(y_test, pred1))

0.9629237288135594
[[45  3]
 [ 3 56]]


In [44]:
title = []
for elt in range(X_train.shape[1]):
    title.append('c'+str(elt))
current_max = roc_auc_score(y_test, pred)
# df_m = pd.DataFrame(np.concatenate((X_test, X_val), axis = 0), columns = title)
df_m = pd.DataFrame(X_val, columns = title)
df_m['target'] = pred2
df_m = df_m.loc[-1:,:]

print(df_m.shape)
for i in range(200):
#     df_ch = pd.DataFrame(np.concatenate((X_test, X_val), axis = 0), columns = title)
    df_ch = pd.DataFrame(X_val, columns = title)
    df_ch['target'] = pred2

    df_ch1 = df_ch.sample(frac = 0.01)

    X1 = df_ch1[title].values
    y1 = df_ch1['target'].values
    
    clf = QuadraticDiscriminantAnalysis(reg_param = 0.11)
#     clf = NuSVC(probability=True, kernel='poly', degree=2, gamma='auto', random_state=2, nu=0.7, coef0=0.08)

    clf.fit(np.concatenate((X_train, X1), axis = 0), np.concatenate((y_train, y1), axis = 0))  


    pred = clf.predict_proba(X_test)[:,1]
    pred1 = clf.predict(X_test)

#     pred2 = clf.predict(np.concatenate((X_test, X_val), axis = 0))
    pred2 = clf.predict(X_val)
    if roc_auc_score(y_test, pred) > current_max:
        current_max = roc_auc_score(y_test, pred)
        X_train = np.concatenate((X_train, X1), axis = 0)
        y_train = np.concatenate((y_train, y1), axis = 0)
        df_m = pd.concat([df_m, df_ch1])
#     df_m = df_m.drop_duplicates()
    print(roc_auc_score(y_test, pred))
    
print(confusion_matrix(y_test, pred1))

(253, 47)
0.974223163841808
0.9745762711864407
0.9749293785310735
0.9752824858757063
0.9749293785310735
0.9766949152542374
0.9766949152542374
0.9777542372881357
0.9781073446327685
0.9777542372881356
0.9777542372881357
0.9781073446327685
0.9784604519774012
0.9781073446327685
0.9774011299435029
0.9781073446327684
0.9770480225988701
0.980225988700565
0.9798728813559322
0.978813559322034
0.9795197740112995
0.9798728813559322
0.9805790960451978
0.9805790960451978
0.9805790960451978
0.980225988700565
0.9809322033898306
0.9791666666666667
0.9816384180790961
0.9809322033898304
0.9823446327683616
0.9805790960451978
0.9816384180790961
0.9823446327683616
0.9826977401129944
0.9819915254237288
0.9823446327683617
0.9816384180790961
0.9826977401129945
0.9844632768361582
0.9837570621468927
0.9841101694915254
0.9844632768361582
0.9837570621468927
0.9837570621468926
0.9837570621468927
0.9826977401129944
0.9830508474576272
0.9844632768361582
0.983050847457627
0.984816384180791
0.9844632768361582
0.983050

In [47]:
X_t = df_m[title].values
y_t = df_m['target']
clf = QuadraticDiscriminantAnalysis(reg_param = 0.11)
clf.fit(X_t, y_t)
pred = clf.predict_proba(X_train)[:,1]
roc_auc_score(y_train, pred)

0.935889826782731

In [21]:
# df_m = df_m.drop_duplicates(subset = title, keep = False)#.shape
df_m.shape

(255, 47)

In [ ]:
pred = np.zeros(df_test.shape[0])
for i in range(512):
    print(i)
    train =  df_train[df_train['wheezy-copper-turtle-magic']==i]
    test =  df_test[df_test['wheezy-copper-turtle-magic']==i]
    y_train = train['target'].values
    
    train = train[cols]
    test = test[cols]
    
    
    n_features = VarianceThreshold(threshold=1.5).fit_transform(train[cols]).shape[1]
    mapper = PCA(n_components=n_features, whiten = True)
    mapper.fit(np.concatenate((train, test), axis=0))
    train = mapper.fit_transform(train)
    test = mapper.fit_transform(test)
    
    clf = QuadraticDiscriminantAnalysis(reg_param = 0.11)
    
    clf.fit(train, y_train)
    
    pred1 = clf.predict_proba(test)[:,1]
    for count in range(2):
        X1 = test[pred1>0.9999999999999999]
        X0 = test[pred1<0.0000000000000001]
        y1 = np.zeros(X1.shape[0])+1
        y0 = np.zeros(X0.shape[0])
        train = np.concatenate((train, X1, X0))
        y_train = np.concatenate((y_train, y1, y0))
        clf.fit(train, y_train)
        pred1 = clf.predict_proba(test)[:,1]
    
    
    pred[df_test['wheezy-copper-turtle-magic']==i] = clf.predict_proba(test)[:,1]


    
    
    
    
    

In [ ]:
submission['target'] = pred
submission.to_csv('submission.csv', index=False)